In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

try:
    import panel
except ImportError:
    print("Installing panel...")
    !pip install panel

In [3]:
import numpy as np
import torch
import pyvista as pv
from trimesh import Trimesh
from manotorch.manolayer import ManoLayer, MANOOutput
from manotorch.axislayer import AxisLayerFK
from manotorch.anchorlayer import AnchorLayer

pv.start_xvfb()

### Run ManoLayer and Show MANO Hand Mesh

In [4]:
mano_layer = ManoLayer(
    rot_mode="axisang",
    use_pca=False,
    side="right",
    center_idx=None,
    mano_assets_root="../assets/mano",
    flat_hand_mean=False,
)
axis_layer = AxisLayerFK(mano_assets_root="../assets/mano")
anchor_layer = AnchorLayer(anchor_root="../assets/anchor")

BS = 1
random_shape = torch.rand(BS, 10)
root_pose = torch.tensor([[0.0, 1.0, 1.0]]).repeat(BS, 1)
finger_pose = torch.zeros(BS, 45)
hand_pose = torch.cat([root_pose, finger_pose], dim=1)

mano_results: MANOOutput = mano_layer(hand_pose, random_shape)
verts = mano_results.verts
faces = mano_layer.th_faces
V = verts[0].numpy()
F = faces.numpy()
tmesh = Trimesh(V, F)
mesh = pv.wrap(tmesh)

next cell shows MANO hand mesh in its mean pose, you can 
* drag to rotate; 
* press 'w' to show edges;
* press 'v' to show vertices;
* press 's' to show surface

In [5]:
pv.plot(mesh,
        color=[0.8, 0.8, 0.8],
        jupyter_backend='panel',
        opacity=0.6,
        background="white",
        show_edges=False,
        smooth_shading=True)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, orientation_widget=True, sizing_mode='stretch_width')

### Show the Anatomy-Aligned Coordinate System

The back (twist), up (spread), and left (bend) axis in its _local_ anatomy-aligned (**a**) coordinate system are defined as follows:  

$$
    (\begin{array}{ccc}
        b & u & l \\
    \end{array})_a  = 
    \left(
        \begin{array}{ccc}
            1 & 0 & 0 \\
            0 & 1 & 0 \\
            0 & 0 & 1 \\
        \end{array}
    \right)_{a}
$$

Therefore, to acquire these axes in the _global_ system, we left-multiply the $T_{g,a}$ acquired from the axis_layer
$$
(b, u, l)_{g} =  T_{g,a} \cdot (b, u, l)_{a}
$$

In [6]:
T_g_p = mano_results.transforms_abs  # (B, 16, 4, 4)
T_g_a, R, ee = axis_layer(T_g_p)  # ee (B, 16, 3)

bul_axes_loc = torch.eye(3).reshape(1, 1, 3, 3).repeat(BS, 16, 1, 1).to(verts.device)
bul_axes_glb = torch.matmul(T_g_a[:, :, :3, :3], bul_axes_loc)  # (B, 16, 3, 3)

b_axes_dir = bul_axes_glb[:, :, :, 0].numpy()  # bend direction (B, 16, 3)
u_axes_dir = bul_axes_glb[:, :, :, 1].numpy()  # up direction (B, 16, 3)
l_axes_dir = bul_axes_glb[:, :, :, 2].numpy()  # left direction (B, 16, 3)

axes_cen = T_g_a[:, :, :3, 3].numpy()  # center (B, 16, 3)

Now, show the b, u, l axes in the global system.

In [7]:
pl = pv.Plotter(off_screen=False)
pl.add_mesh(mesh, opacity=0.4, name="mesh", smooth_shading=True)
pl.add_arrows(axes_cen, b_axes_dir, color="red", mag=0.02)
pl.add_arrows(axes_cen, u_axes_dir, color="yellow", mag=0.02)
pl.add_arrows(axes_cen, l_axes_dir, color="blue", mag=0.02)
pl.set_background('white')
pl.add_camera_orientation_widget()
pl.show(interactive=True, jupyter_backend='panel')


VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, sizing_mode='stretch_width')

### Show the Anchors

In [9]:
verts = mano_results.verts
anchors = anchor_layer(verts)[0].numpy()

pl = pv.Plotter(off_screen=False)
pl.add_mesh(mesh, opacity=0.4, name="mesh", smooth_shading=True)

n_achors = anchors.shape[0]
for i in range(n_achors):
    pl.add_mesh(pv.Cube(center=anchors[i], x_length=3e-3, y_length=3e-3, z_length=3e-3), color="yellow", name=f"anchor{i}")
pl.set_background('white')
pl.show(interactive=True, jupyter_backend='panel')

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, sizing_mode='stretch_width')